In [1]:
import flask

import numpy as np
import pandas as pd
import pickle
from collections import defaultdict

from sklearn.neighbors import NearestNeighbors



from fuzzywuzzy import process
from fuzzywuzzy import fuzz



In [2]:
with open('./data/df_merged_cleaned.pkl', 'rb') as picklefile:
    df_merged = pickle.load(picklefile)

with open('./data/topic_names.pkl', 'rb') as picklefile:
    topic_names = pickle.load(picklefile)

with open('./data/lda_model.pkl', 'rb') as picklefile:
    lda_mod = pickle.load(picklefile)

with open('./data/vectorizer.pkl', 'rb') as picklefile:
    vect_mod = pickle.load(picklefile)

with open('./data/lda_model_data.pkl', 'rb') as picklefile:
    lda_data = pickle.load(picklefile)

cleaned_talks = df_merged['text']

titles = df_merged['title']

In [3]:
app = flask.Flask(__name__)

In [ ]:
def get_recommendations(first_article, model, vectorizer, training_vectors,title, ind):

    new_vec = model.transform(
        vectorizer.transform([first_article]))

    nn = NearestNeighbors(n_neighbors=10, metric='cosine', algorithm='brute')
    nn.fit(training_vectors)
    results = nn.kneighbors(new_vec)
    recommend_list = results[1][0]
    scores = results[0]

    rec_dict = defaultdict(list)
    urls = []
    ss = np.array(scores).flat
    
    print(recommend_list)
#     for i, resp in enumerate(recommend_list):
        
#         rec_dict["0"].append(df_merged.iloc[resp,1])
        
# #         rec_dict['0'].append(df_merged.iloc[i]['title'])
# #         rec_dict['0'].append(df_merged.iloc[i]['url'])
# #         rec_dict['0'].append(df_merged.iloc[i]['title'])
        
# #         rec_dict["desc"].append(df_merged.iloc[resp,1])
# #         rec_dict["url"].append(df_merged.iloc[i]['url'])
# #         rec_dict["title"].append(df_merged.iloc[i]['title'])
# #         urls.append(df_merged.iloc[i]['url'])
# #         rec_dict['title'] = df_merged[title]
    
#     for i, resp in enumerate(recommend_list):
#         rec_dict['0'].append(df_merged.iloc[i]['url'])

    for i in recommend_list:
        rec_dict['0'].append(df_merged.iloc[i]['title'])
        
    for i in recommend_list:
        rec_dict['0'].append(df_merged.iloc[i]['url'])
        
        
    return rec_dict

In [ ]:
get_recommendations(cleaned_talks[7], lda_mod, vect_mod,
                                       lda_data, 'Do schools kill creativity?', 1)

[   7 2037  359 1529 1748  358 2385 1636 1851 1522]


defaultdict(list,
            {'0': ["Behind the design of Seattle's library",
              'Why great architecture should tell a story',
              'Ways of seeing',
              'Architecture at home in its community',
              'How to reinvent the apartment building',
              'Organic algorithms in architecture',
              'Why glass towers are bad for city life -- and what we need instead',
              'How public spaces make cities work',
              'Why the buildings of the future will be shaped by ... you',
              'Ingenious homes in unexpected places',
              'https://www.ted.com/talks/joshua_prince_ramus_on_seattle_s_library\n',
              'https://www.ted.com/talks/ole_scheeren_why_great_architecture_should_tell_a_story\n',
              'https://www.ted.com/talks/rob_forbes_on_ways_of_seeing\n',
              'https://www.ted.com/talks/xavier_vilalta_architecture_at_home_in_its_community\n',
              'https://www.ted.com/talks/m

In [ ]:
@app.route("/")
def visualize_page():

    
    with open("ted_html.html", 'r') as viz_file:
        return viz_file.read()

In [ ]:
@app.route("/ted", methods=["POST"])
def ted():
    """
    When A POST request with json data is made to this uri,
    Read the example from the json, predict probability and
    send it with a response
    """
    # Get decision score for our example that came with the request
    data = flask.request.json
    print(data)
    X = data#["example"])
    #X = str(X[0])
    title, score, talk_ind = process.extractOne(X, titles, scorer=fuzz.token_set_ratio)
    recs = get_recommendations(cleaned_talks[talk_ind],lda_mod, vect_mod, lda_data, title,talk_ind)
#     recs, urls = get_recommendations(cleaned_talks[talk_ind],lda_mod, vect_mod, lda_data, title,talk_ind)
    print(recs.values())
    # Put the result in a nice dict so we can send it as json
    #results = {"recommends": recs[1]}
    #print(results)
    
    return flask.jsonify(recs['0'])

    

#     return flask.jsonify(recs['0']), flask.jsonify(urls)
#     return flask.jsonify(recs['0']), flask.jsonify(recs['1'])

#     print(flask.jsonify(descriptions = recs['desc'], titles = recs['title'], urls = recs['urls']))
    
#     return flask.jsonify(descriptions = recs['desc'], titles = recs['title'], urls = recs['urls'])

#     d = {'descriptions' : recs['desc'], 'titles' : recs['title'], 'urls' : recs['urls']}
#     message = {
#         'status': 200,
#         'message': 'OK',
#         'scores': d
#     }
#     resp = flask.jsonify(message)
#     resp.status_code = 200
#     return resp
app.run(host='0.0.0.0')
app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:395: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = effective_n_jobs(self.n_jobs)
127.0.0.1 - - [27/May/2019 12:52:49] "POST /ted HTTP/1.1" 200 -


educate
[ 129 1911   39  292 1704 1184  965 2226 2082 2213]
dict_values([['How to educate leaders? Liberal arts', "This is what it's like to go undercover in North Korea", 'My wish: Three actions for Africa', 'The psychology of evil', 'When people of Muslim heritage challenge fundamentalism', 'The shared wonder of film', 'Pay attention to nonviolence', 'Inside the mind of a former radical jihadist', "A conservative's plea: Let's work together", 'Immigrant voices make democracy stronger', 'https://www.ted.com/talks/patrick_awuah_on_educating_leaders\n', 'https://www.ted.com/talks/suki_kim_this_is_what_it_s_like_to_go_undercover_in_north_korea\n', 'https://www.ted.com/talks/bono_s_call_to_action_for_africa\n', 'https://www.ted.com/talks/philip_zimbardo_on_the_psychology_of_evil\n', 'https://www.ted.com/talks/karima_bennoune_the_side_of_terrorism_that_doesn_t_make_headlines\n', 'https://www.ted.com/talks/beeban_kidron_the_shared_wonder_of_film\n', 'https://www.ted.com/talks/julia_bacha\n'

/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:395: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = effective_n_jobs(self.n_jobs)
127.0.0.1 - - [27/May/2019 12:52:57] "POST /ted HTTP/1.1" 200 -


do schools
[   0  406 1342 1354 1120 1372 2416 1723 2247 1968]
dict_values([['Do schools kill creativity?', 'The difference between winning and succeeding', 'Kids need structure', 'Dare to educate Afghan girls', 'Unlock the intelligence, passion, greatness of girls', 'A girl who demanded school', 'Can art amend history?', 'Why lunch ladies are heroes', 'Help for kids the education system ignores', 'How I stopped the Taliban from shutting down my school', 'https://www.ted.com/talks/ken_robinson_says_schools_kill_creativity\n', 'https://www.ted.com/talks/john_wooden_on_the_difference_between_winning_and_success\n', 'https://www.ted.com/talks/colin_powell_kids_need_structure\n', 'https://www.ted.com/talks/shabana_basij_rasikh_dare_to_educate_afghan_girls\n', 'https://www.ted.com/talks/leymah_gbowee_unlock_the_intelligence_passion_greatness_of_girls\n', 'https://www.ted.com/talks/kakenya_ntaiya_a_girl_who_demanded_school\n', 'https://www.ted.com/talks/titus_kaphar_can_art_amend_history\n',

...

do schools
[   0  406 1342 1354 1120 1372 2416 1723 2247 1968]
dict_values([['Do schools kill creativity?', 'The difference between winning and succeeding', 'Kids need structure', 'Dare to educate Afghan girls', 'Unlock the intelligence, passion, greatness of girls', 'A girl who demanded school', 'Can art amend history?', 'Why lunch ladies are heroes', 'Help for kids the education system ignores', 'How I stopped the Taliban from shutting down my school', 'https://www.ted.com/talks/ken_robinson_says_schools_kill_creativity\n', 'https://www.ted.com/talks/john_wooden_on_the_difference_between_winning_and_success\n', 'https://www.ted.com/talks/colin_powell_kids_need_structure\n', 'https://www.ted.com/talks/shabana_basij_rasikh_dare_to_educate_afghan_girls\n', 'https://www.ted.com/talks/leymah_gbowee_unlock_the_intelligence_passion_greatness_of_girls\n', 'https://www.ted.com/talks/kakenya_ntaiya_a_girl_who_demanded_school\n', 'https://www.ted.com/talks/titus_kaphar_can_art_amend_history\n',

/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:395: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = effective_n_jobs(self.n_jobs)
127.0.0.1 - - [27/May/2019 12:57:28] "POST /ted HTTP/1.1" 200 -


best stats
[   4  493 1586 1214  547  440 1754  513 1963 1982]
dict_values([["The best stats you've ever seen", 'Let my dataset change your mindset', 'Why smart statistics are the key to fighting crime', "Let's crowdsource the world's goals", "Asia's rise -- how and when", "Ideas for India's future", 'New thoughts on capital in the twenty-first century', 'Mapping the future of countries', 'How fear drives American politics', 'How we can make the world a better place by 2030', 'https://www.ted.com/talks/hans_rosling_shows_the_best_stats_you_ve_ever_seen\n', 'https://www.ted.com/talks/hans_rosling_at_state\n', 'https://www.ted.com/talks/anne_milgram_why_smart_statistics_are_the_key_to_fighting_crime\n', 'https://www.ted.com/talks/jamie_drummond_how_to_set_goals_for_the_world\n', 'https://www.ted.com/talks/hans_rosling_asia_s_rise_how_and_when\n', 'https://www.ted.com/talks/nandan_nilekani_s_ideas_for_india_s_future\n', 'https://www.ted.com/talks/thomas_piketty_new_thoughts_on_capital_in_

/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:395: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = effective_n_jobs(self.n_jobs)
127.0.0.1 - - [27/May/2019 12:57:28] "POST /ted HTTP/1.1" 200 -


[   4  493 1586 1214  547  440 1754  513 1963 1982]
dict_values([["The best stats you've ever seen", 'Let my dataset change your mindset', 'Why smart statistics are the key to fighting crime', "Let's crowdsource the world's goals", "Asia's rise -- how and when", "Ideas for India's future", 'New thoughts on capital in the twenty-first century', 'Mapping the future of countries', 'How fear drives American politics', 'How we can make the world a better place by 2030', 'https://www.ted.com/talks/hans_rosling_shows_the_best_stats_you_ve_ever_seen\n', 'https://www.ted.com/talks/hans_rosling_at_state\n', 'https://www.ted.com/talks/anne_milgram_why_smart_statistics_are_the_key_to_fighting_crime\n', 'https://www.ted.com/talks/jamie_drummond_how_to_set_goals_for_the_world\n', 'https://www.ted.com/talks/hans_rosling_asia_s_rise_how_and_when\n', 'https://www.ted.com/talks/nandan_nilekani_s_ideas_for_india_s_future\n', 'https://www.ted.com/talks/thomas_piketty_new_thoughts_on_capital_in_the_twenty_